In [1]:
# import talib
import numpy as np
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from ciso8601 import parse_datetime

from ftx_client import FtxClient

## API Init

In [2]:
api_key = '4MZCayt-zZKp3QQFOBgPFdwOK5B1zoxJPMeFAGKz'
api_secret = 'cfQxfctS5j7lR0iq-iGsiItt4znrH9P4AkF-IzMH'

client = FtxClient(api_key=api_key, api_secret=api_secret)

## Variables

In [3]:
resolution = 1*60 # 15 minutes in seconds

startTime = None
endTime = None

market = 'BTC-PERP'

## HA Computation

In [4]:
past_candles = 1500

# utc_offset = time.timezone if (time.localtime().tm_isdst == 0) else time.altzone
# utc_offset = utc_offset / 60 / 60 * -1
# print('utc_offset: ', utc_offset)

# # Use only candles from the past x hours

# past_hours = 24
# utc_time = datetime.now() - timedelta(hours=(utc_offset))
# print('utc time: ', utc_time)

# start_datetime = utc_time - timedelta(hours = past_hours)
# start_timestamp = (start_datetime - datetime(1970, 1, 1)) / timedelta(seconds=1)
# print('start_timestamp: ', start_timestamp)
# end_timestamp = (utc_time - datetime(1970, 1, 1)) / timedelta(seconds=1)
# # candles = candles[-int(past_hours * (3600.0 / resolution)):]
# candles = client.get_historical_candles(market, resolution, start_timestamp, end_timestamp)

candles = client.get_candles(market, resolution)[-past_candles:]


N = len(candles)

print(N)
print(candles[-5:])

1500
[{'close': 29045.0, 'high': 29046.0, 'low': 29045.0, 'open': 29046.0, 'startTime': '2020-12-31T05:52:00+00:00', 'time': 1609393920000.0, 'volume': 252354.11395}, {'close': 29032.0, 'high': 29045.5, 'low': 29030.0, 'open': 29045.0, 'startTime': '2020-12-31T05:53:00+00:00', 'time': 1609393980000.0, 'volume': 611225.3385}, {'close': 29028.0, 'high': 29032.0, 'low': 29005.5, 'open': 29032.0, 'startTime': '2020-12-31T05:54:00+00:00', 'time': 1609394040000.0, 'volume': 1685369.33885}, {'close': 28987.5, 'high': 29028.0, 'low': 28987.0, 'open': 29028.0, 'startTime': '2020-12-31T05:55:00+00:00', 'time': 1609394100000.0, 'volume': 349425.96355}, {'close': 28999.0, 'high': 29005.5, 'low': 28987.5, 'open': 28987.5, 'startTime': '2020-12-31T05:56:00+00:00', 'time': 1609394160000.0, 'volume': 109581.4662}]


In [5]:
ha_c = []
ha_o = []
ha_high = []
ha_low = []

ha_c.append(sum([candles[0][x] for x in ['high', 'low', 'open', 'close']]) / 4)
ha_o.append((candles[0]['open'] + candles[0]['close']) / 2)
ha_high.append(candles[0]['high'])
ha_low.append(candles[0]['low'])

for i in range(1, N):
    candle = candles[i]
    
    ha_c.append(sum([candle[x] for x in ['high', 'low', 'open', 'close']]) / 4)
    ha_o.append((ha_c[i-1] + ha_o[i-1]) / 2)
    ha_high.append(max([candle['high'], ha_c[i], ha_o[i]]))
    ha_low.append(min([candle['low'], ha_c[i], ha_o[i]]))
    
print(len(ha_c))
print(ha_c[:10])

1500
[27896.875, 27913.625, 27924.25, 27944.625, 27967.125, 27925.5, 27888.0, 27879.875, 27875.375, 27883.625]


## Backtest criterions

In [6]:
def buy_criterion(prices):
    """
    prices -> prices till now, last price is most current candle
    """

    condition = prices[-1] > prices[-2]
    
    if condition:
        return True
    return False

def sell_criterion(prices):
    """
    prices -> prices till now, last price is most current candle
    """

    condition = prices[-1] < prices[-2]
    
    if condition:
        return True
    return False

## Backtest

In [7]:
balance = {'USD': 1000.0, 'BTC': 0.0}
min_criterion_candles = 2 # Minimum number of candles in the past which are required for buy/sell criterions.
                          # Includes current candle

activeLong = False
numTrades = 0

for t in range(min_criterion_candles - 1, N):
    if not activeLong and buy_criterion(ha_c[:(t + 1)]):
        print('Entering long at ha_close = ', ha_c[t])
        print('Old balance = {}\n'.format(balance['USD']))
        
        activeLong = True
        balance['BTC'] = balance['USD'] / ha_c[t]
        balance['USD'] = 0.0
        
    if activeLong and sell_criterion(ha_c[:(t + 1)]):
        print('Exiting long at ha_close = ', ha_c[t])
        
        newBalance = balance['BTC'] * ha_c[t]
        
        print('New balance = {}\n'.format(newBalance))
        
        activeLong = False
        balance['USD'] = newBalance
        balance['BTC'] = 0.0
        numTrades += 1
        
final_usd = balance['USD']
if final_usd == 0.0:
    final_usd = balance['BTC'] * ha_c[-1]
        
print('final balance: ', final_usd)
print('number of trades: ', numTrades)

Entering long at ha_close =  27913.625
Old balance = 1000.0

Exiting long at ha_close =  27925.5
New balance = 1000.4254194860038

Entering long at ha_close =  27883.625
Old balance = 1000.4254194860038

Exiting long at ha_close =  27996.875
New balance = 1004.4886709017285

Entering long at ha_close =  27992.875
Old balance = 1004.4886709017285

Exiting long at ha_close =  27975.0
New balance = 1003.8472492902517

Entering long at ha_close =  27971.75
Old balance = 1003.8472492902517

Exiting long at ha_close =  27968.375
New balance = 1003.7261276419332

Entering long at ha_close =  27950.75
Old balance = 1003.7261276419332

Exiting long at ha_close =  27944.875
New balance = 1003.5151533031445

Entering long at ha_close =  27929.75
Old balance = 1003.5151533031445

Exiting long at ha_close =  28014.0
New balance = 1006.5422534979472

Entering long at ha_close =  27994.75
Old balance = 1006.5422534979472

Exiting long at ha_close =  28121.5
New balance = 1011.0995090773276

Entering 